## ReAct Prompting
First introduced in a paper by [Yao et al., 2022](https://arxiv.org/abs/2210.03629), ReAct a reasoning and acting paradigm that guides LLM to respond in a structured manager to complex queries. Reasoning and actions are interleaved and progressive so that LLM progresses from one result to another, using the previous answer.

<img src="https://www.promptingguide.ai/_next/image?url=%2F_next%2Fstatic%2Fmedia%2Freact.8e7c93ae.png&w=1920&q=75">

Results suggest that ReAct outperforms other leading methods in language and decision-making tasks, enhances human understanding and trust in large language models (LLMs), and works best when combined with chain-of-thought (CoT), utilizing both internal knowledge and external information during reasoning.

Let's first use a few-shot examples to indicate how an LLM should progress with its reasoning and actiong to solve a complex query, requiring intermideiate results to progress to its final outcome



In [1]:
import warnings
import os

import openai
from openai import OpenAI

from dotenv import load_dotenv, find_dotenv

Load the environment

In [2]:
_ = load_dotenv(find_dotenv()) # read local .env file
warnings.filterwarnings('ignore')
openai.api_base = os.getenv("ANYSCALE_API_BASE", os.getenv("OPENAI_API_BASE"))
openai.api_key = os.getenv("ANYSCALE_API_KEY", os.getenv("OPENAI_API_KEY"))
MODEL = os.getenv("MODEL")
print(f"Using MODEL={MODEL}; base={openai.api_base}")

Using MODEL=gpt-4-1106-preview; base=https://api.openai.com/v1


In [3]:
# Create the OpenAI client, which can be used transparently with Anyscale 
#Endpoints too

from openai import OpenAI

client = OpenAI(
    api_key = openai.api_key,
    base_url = openai.api_base
)

In [4]:
# Utility function to send and fetch response

def get_commpletion(clnt: object, model: str, system_content: str, user_content:str) -> str:
    chat_completion = clnt.chat.completions.create(
        model=model,
    messages=[{"role": "system", "content": system_content},
              {"role": "user", "content": user_content}],
    temperature = 0.8)

    response = chat_completion.choices[0].message.content
    return response

### Example 1: Use a in-context example for calculating an age of a person using its existing knowledge base
An example use case for a few-shot learning is to provide an example pattern of chain of though that the model can learn from and generalize how to use ReAct. Using what it has learned as an in-context example, it can generate a response for its subsequent prompt in the manner suggested.

We will use an example from this [guide](https://www.promptingguide.ai/techniques/react).

In [5]:
system_content = """
You are supreme repository of knowledge and can answer
questions by deliberating following an thought, action, and
observation pattern known as ReAct.
"""

In [6]:
react_user_prompt = """
Question: What is the elevation range for the area that the eastern sector of the
Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector
of the Colorado orogeny extends into, then find the elevation range of the
area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in
Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern
sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called
the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I
need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[High Plains (United States)]
Observation 4: The High Plains are a subregion of the Great Plains. From east to west, the
High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130
m).[3]
Thought 5: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer
is 1,800 to 7,000 ft.
Action 5: Finish[1,800 to 7,000 ft]

Question: Based on information provided to you upto 2023, Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?
...
"""

In [7]:
response = get_commpletion(client, MODEL, system_content, react_user_prompt)
print(f"{response}\n")

Based on the information available up to 2023, it's important to note that personal relationships can change frequently and private details about individuals are not always publicly disclosed or may be outdated. As of my last update, Olivia Wilde had been in a relationship with Harry Styles, but whether they are still together or whether she has a new boyfriend may have changed since then.

To answer the second part of the question, if we assume that we're referring to Harry Styles and we're using his age as of 2023, Harry Styles was born on February 1, 1994. This would make him 29 years old in 2023. Raising 29 to the power of 0.23 can be calculated as follows:

Thought 1: I need to calculate 29 raised to the power of 0.23.
Action 1: Calculate[29^0.23]

Since I am unable to perform real-time calculations, you would need to use a calculator or a computational tool to find the exact value of 29 raised to the power of 0.23. The formula to use would be:

\[ 29^{0.23} \]

Performing this ca

### Example 2: Use a in-context example for calculating an age of a person using its existing knowledge base
An example use case for a few-shot learning is to provide an example pattern of chain of though that the model can learn from and generalize how to use ReAct. Using what it has learned as an in-context example, it can generate a response for its subsequent prompt in the manner suggested.

We will use an example from this [guide](https://www.promptingguide.ai/techniques/react).

In [8]:
react_user_prompt_2 = """
Question: What is the elevation range for the area that the eastern sector of the
Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector
of the Colorado orogeny extends into, then find the elevation range of the
area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in
Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern
sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called
the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I
need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[High Plains (United States)]
Observation 4: The High Plains are a subregion of the Great Plains. From east to west, the
High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130
m).[3]
Thought 5: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer
is 1,800 to 7,000 ft.
Action 5: Finish[1,800 to 7,000 ft]

Question: Based on information provided to you upto 2023, aside from the Apple Remote, what other devices can 
control the program Apple Remote was originally designed to interact with?
...
"""

In [9]:
response = get_commpletion(client, MODEL, system_content, react_user_prompt_2)
print(f"{response}\n")

Thought 1: Apple Remote was originally designed to interact with various Apple devices, such as Mac computers and the Apple TV. To answer this question accurately, I need to consider the different devices that can control these Apple products.

Action 1: Recall[Devices compatible with Mac and Apple TV]

Observation 1: Devices that can control Mac computers and Apple TVs include other Apple devices with the Remote app installed (such as iPhones, iPads, and iPod Touches), third-party universal remotes that are programmable to work with Apple TV, and keyboards or mice when it comes to Macs.

Thought 2: Besides the Apple Remote, there are several options for controlling Apple devices. iPhones, iPads, and iPod Touches can use the Remote app. Macs can be controlled with keyboards and mice, and programmable universal remotes can be used with Apple TV.

Action 2: Combine[Devices compatible with Mac and Apple TV]

Observation 2: The combination of compatible devices includes Apple's iOS devices

### Example 3: Use a in-context example for calculating an age of a person using its existing knowledge base
An example use case for a few-shot learning is to provide an example pattern of chain of though that the model can learn from and generalize how to use ReAct. Using what it has learned as an in-context example, it can generate a response for its subsequent prompt in the manner suggested.

We will use an example from this [guide](https://www.promptingguide.ai/techniques/react).

In [10]:
react_user_prompt_3 = """
Question: What is the elevation range for the area that the eastern sector of the
Colorado orogeny extends into?
Thought 1: I need to search Colorado orogeny, find the area that the eastern sector
of the Colorado orogeny extends into, then find the elevation range of the
area.
Action 1: Search[Colorado orogeny]
Observation 1: The Colorado orogeny was an episode of mountain building (an orogeny) in
Colorado and surrounding areas.
Thought 2: It does not mention the eastern sector. So I need to look up eastern
sector.
Action 2: Lookup[eastern sector]
Observation 2: (Result 1 / 1) The eastern sector extends into the High Plains and is called
the Central Plains orogeny.
Thought 3: The eastern sector of Colorado orogeny extends into the High Plains. So I
need to search High Plains and find its elevation range.
Action 3: Search[High Plains]
Observation 3: High Plains refers to one of two distinct land regions
Thought 4: I need to instead search High Plains (United States).
Action 4: Search[High Plains (United States)]
Observation 4: The High Plains are a subregion of the Great Plains. From east to west, the
High Plains rise in elevation from around 1,800 to 7,000 ft (550 to 2,130
m).[3]
Thought 5: High Plains rise in elevation from around 1,800 to 7,000 ft, so the answer
is 1,800 to 7,000 ft.
Action 5: Finish[1,800 to 7,000 ft]

Question: Based on information provided to you upto 2023, how do you calculate the value
of PI. With its current value of PI, given a circle of diameter 2 meters, what its its 
circumcernce and area?
...
"""

In [11]:
response = get_commpletion(client, MODEL, system_content, react_user_prompt_3)
print(f"{response}\n")

Thought 1: To calculate the value of pi (π), mathematicians use various algorithms and numerical methods because π is an irrational number and cannot be expressed as a simple fraction. Historically, π has been approximated by methods like polygon approximation or series expansions.

Action 1: No action required here because the value of π is already known and accepted as approximately 3.14159.

Observation 1: The value of π is widely known to be approximately 3.14159.

Thought 2: To find the circumference of a circle, the formula is C = πd, where d is the diameter, and π is approximated as 3.14159. To find the area of a circle, the formula is A = πr^2, where r is the radius (half of the diameter).

Action 2: Calculate circumference and area using the known value of π.

Observation 2: For a circle with a diameter of 2 meters, the radius would be 1 meter (half of the diameter).

Circumference calculation:
C = πd = π * 2 = 3.14159 * 2 ≈ 6.28318 meters

Area calculation:
A = πr^2 = π * 1^2

## All this is amazing! 😜 Feel the wizardy in prompt power 🧙‍♀️